# Рекомендация тарифов для клиентов мобильного оператора

<h1>Оглавление проекта<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта-" data-toc-modified-id="Описание-проекта--1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта <a name="1"></a></a></span></li><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Знакомство-с-данными-" data-toc-modified-id="Знакомство-с-данными--3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Знакомство с данными <a name="2"></a></a></span></li><li><span><a href="#Разбиение-данных-на-выборки-" data-toc-modified-id="Разбиение-данных-на-выборки--4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Разбиение данных на выборки <a name="3"></a></a></span></li><li><span><a href="#Выбор-модели-и-подбор-гиперпараметров-" data-toc-modified-id="Выбор-модели-и-подбор-гиперпараметров--5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выбор модели и подбор гиперпараметров <a name="4"></a></a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверка-качества-модели-" data-toc-modified-id="Проверка-качества-модели--6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Проверка качества модели <a name="5"></a></a></span></li><li><span><a href="#Проверка-модели-на-адекватность-" data-toc-modified-id="Проверка-модели-на-адекватность--7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Проверка модели на адекватность <a name="6"></a></a></span></li><li><span><a href="#Общий-вывод-" data-toc-modified-id="Общий-вывод--8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Общий вывод <a name="7"></a></a></span></li></ul></div>

## Описание проекта <a name="1"></a>
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали ранее.

Построим модель с максимально большим значением *accuracy*. Доведем долю правильных ответов по крайней мере до 0.75. Проверим *accuracy* на тестовой выборке.

## Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Знакомство с данными <a name="2"></a>

Импортируем необходимые для работы библиотеки, классы и функции.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from random import choice

Откроем файл с данными и изучим его. Путь к файлу: datasets/users_behavior.csv

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Посмотрим на основные харктеристики набора данных.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Загрузили сведения о том, сколько каждый из пользователь использовал услуг в течении месяца и какой у него был при этом тариф. Видим, что для данных выбраны неоптимальные типы. Пропусков нет. 

Кроме того, обратим внимание на то, что количество звонков и общая продолжительность звонков не являются полностью независимыми. Чем больше звоноков, тем больше длительность. Нужно изучить вопрос мультиколлинеарности этих признаков.

Преобразуем типы. Для этого нам понадобится знать максимальные значения.

In [4]:
df.max()

calls         244.00
minutes      1632.06
messages      224.00
mb_used     49745.73
is_ultra        1.00
dtype: float64

In [5]:
dict_types = {
            'calls': 'uint8',
            'minutes': 'float16',
            'messages': 'uint8',
            'mb_used': 'float16',
            'is_ultra': 'uint8'
             }
df = df.astype(dict_types)

Посмотрим на коэффициент корреляции Пирсона в столбцах calls и minutes.

In [6]:
df['calls'].corr(df['minutes'])

0.9820855289609572

Корреляция очень высокая. Оставим в наборе только один из признаков: minutes.

In [7]:
del df['calls']

## Разбиение данных на выборки <a name="3"></a>

Разделим исходные данные на обучающую, валидационную и тестовую выборки. Сделаем это в соотношении 8:1:1. Случайное состояние выберем 0 и дальше последовательно будем его использовать. 

In [8]:
X, y = df[df.columns[:-1]], df[df.columns[-1]]

X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.2, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_, y_, test_size=0.5, random_state=0)

## Выбор модели и подбор гиперпараметров <a name="4"></a>

Так как наша задача относится к задаче классификации, то в качестве моделей будем выбирать классификаторы. Изучим 3 модели: решающее дерево, случайный лес и логистическую регрессию. Исследуем качество разных моделей, меняя гиперпараметры. Выберем модель с лучшими показателями точности предсказания на валидационной выборке.

In [9]:
model_params = {
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': range(2, 5),
            'min_samples_split': [2, 3, 4],
            'criterion': ['gini', 'entropy']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(random_state=0),
        'params': {
            'n_estimators': [10, 25, 40],
            'min_samples_leaf': [5, 6, 7]
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto', random_state=0),
        'params': {
            'C': [1, 5, 20],
            'tol': [1e-4, 1e-5]
        }
    }
}

scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train, y_train)
    prediction_valid = clf.predict(X_valid)
    valid_accuracy = accuracy_score(y_valid, prediction_valid)
    scores.append({
        'model': model_name,
        'best_score': f'{clf.best_score_:.2f}',
        'valid_score': f'{valid_accuracy:.2f}',
        'best_params': clf.best_params_
    })
    
scores = pd.DataFrame(scores, columns=['model','best_score', 'valid_score', 'best_params'])
scores.sort_values(by='best_score', ascending=False)

,model,best_score,valid_score,best_params
1,random_forest,0.81,0.84,"{'min_samples_leaf': 7, 'n_estimators': 40}"
0,decision_tree,0.80,0.82,"{'criterion': 'entropy', 'max_depth': 3, 'min_..."
2,logistic_regression,0.74,0.79,"{'C': 1, 'tol': 1e-05}"


### Вывод

Лучшей моделью оказался случайный лес с параметрами: {'min_samples_leaf': 7, 'n_estimators': 40}. Валидационная точность 84%.

<font color='blue'>Здесь замечу, что метод `GridSearchCV` содержит в себе встроенную *кросс-валидацию* (подробнее о ней в последующих курсах), так что необходимости считать результаты на валидационной выбоке у нас в общем-то нет.  
Также рекомендую не стесняться и попробовать большее количество деревьев, например до 100-150 с шагом 5 или 10, а параметр С в логисточеской регресии рекомендуется выбирать из степеней 10: 0.01, 0.1, 1, 10 и т.д.

In [10]:
model = RandomForestClassifier(min_samples_leaf=7, n_estimators=40, random_state=0)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=7, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

## Проверка качества модели <a name="5"></a>

Проверим качество модели на тестовой выборке.

In [11]:
prediction_test = model.predict(X_test)
accuracy = accuracy_score(y_test, prediction_test)
print(f'Метрика качества точность: {accuracy:.1%}')

Метрика качества точность: 77.6%


Результат презвошел цель в 75% accuracy.

## Проверка модели на адекватность <a name="6"></a>

Проверим модель на вменяемость. Сравним результаты предсказания нашей модели со случайными значениями.

In [12]:
prediction_random = [choice([0,1]) for _ in range(len(y_test))]
random_accuracy = accuracy_score(y_test, prediction_random)
f'Предсказания нашей модели лучше случайных на {accuracy-random_accuracy:.1%}'

'Предсказания нашей модели лучше случайных на 31.4%'

Действительно, наша модель лучше предсказывает, чем случайность. Наша модель показывает вменяемые результаты.

## Общий вывод <a name="7"></a>

Мы загрузили данные, привели типы к более оптимальным. Обнаружили сильную корреляционную связь (98%) между столбцами c общей длительностью звонков и количеством звоноков. Убрали из набора данных количество звоноков, чтобы не иметь дело с мультиколлинеарными признаками. Разбили набор на три части в пропорции 8:1:1. Большую часть использовали для обучения. Две другие для валидации и теста.
В поисках лучшей модели для обучения мы попробовали 3 алгоритма: решающее дерево, случайный лес, логистическая регрессия. Лучшие результаты с проверкой на валидационном наборе, показала модель случайного леса. При этом оптимальные гиперпараметры оказались: min_samples_leaf=7, n_estimators=40.
Проверка качества модели по метрике accuracy на тестовом наборе показала результат 77.6%, что оказалось выше, чем мы ставили себе в качестве цели в 75%.

Полученная нами модель может быть использована для рекомендации оптимального тарифа для клиентов.